In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../src")

In [3]:
import logging
logger = logging.getLogger()
logger.setLevel("DEBUG")

In [4]:
import torchChar

../src\torchChar\prepare_data.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [7]:
dev_dataset = torchChar.load_and_cache_features("dev")

INFO:root:load train_examples/char_examples_Ming.pkl
INFO:root:load train_examples/char_examples_Kai.pkl
INFO:root:load train_examples/char_examples_Yen.pkl
INFO:root:load train_examples/char_examples_FangSong.pkl


In [8]:
len(dev_dataset)

4733

In [9]:
from torch.utils.data import DataLoader

In [11]:
loader = DataLoader(dev_dataset, batch_size=3)

In [12]:
batch = next(iter(loader))

In [14]:
bitmaps = batch[0]
radicals = batch[1]
consonants = batch[2]
vowels = batch[3]
tones = batch[4]

In [67]:
tuple(bitmaps.shape[-2:]) == (64,64)

True

In [47]:
import torch.nn as nn
import torch.nn.functional as F

In [55]:
80*16*16

20480

In [59]:
nradicals = len(torchChar.Radicals())
ntones = len(torchChar.Tones())
nconsonants = len(torchChar.Consonants())
nvowels = len(torchChar.Vowels())

In [58]:
len(radicals.radicals)

214

In [60]:
conv1 = nn.Conv2d(1, 20, 3, padding=1)
pool1 = nn.MaxPool2d(2, 2)
conv2 = nn.Conv2d(20, 40, 5, padding=2)
pool2 = nn.MaxPool2d(2, 2)
conv3 = nn.Conv2d(40, 80, 5, padding=2)

fc1 = nn.Linear(80*16*16, 1000)
fc2 = nn.Linear(1000, 500)
fc3 = nn.Linear(500, 200)
fc_radicals = nn.Linear(500, nradicals)
fc_tones = nn.Linear(200, ntones)
fc_consonants = nn.Linear(200, nconsonants)
fc_vowels = nn.Linear(200, nvowels)

In [63]:
x = bitmaps.unsqueeze(1).float()
h = conv1(x)
print("conv1 shape: " + str(h.shape))
h = F.relu(pool1(h))
print("pool1 shape: " + str(h.shape))
h = conv2(h)
print("conv2 shape: " + str(h.shape))
h = F.relu(pool2(h))
print("pool2 shape: " + str(h.shape))
h = conv3(h)
print("conv3 shape: " + str(h.shape))

v1 = fc1(F.relu(h.view(-1, 80*16*16)))
v1 = fc2(F.relu(v1))

print("v1 shape: " + str(v1.shape))
o_radicals = fc_radicals(F.relu(v1))
print("o_radicals shape: " + str(o_radicals.shape))

v2 = fc3(F.relu(v1))
print("v2 shape: " + str(v2.shape))
o_tones = fc_tones(F.relu(v2))
print("o_tones shape: " + str(o_tones.shape))
o_consonants = fc_consonants(F.relu(v2))
print("o_consonants shape: " + str(o_consonants.shape))
o_vowels = fc_vowels(F.relu(v2))
print("o_vowels shape: " + str(o_vowels.shape))

conv1 shape: torch.Size([3, 20, 64, 64])
pool1 shape: torch.Size([3, 20, 32, 32])
conv2 shape: torch.Size([3, 40, 32, 32])
pool2 shape: torch.Size([3, 40, 16, 16])
conv3 shape: torch.Size([3, 80, 16, 16])
v1 shape: torch.Size([3, 500])
o_radicals shape: torch.Size([3, 214])
v2 shape: torch.Size([3, 200])
o_tones shape: torch.Size([3, 5])
o_consonants shape: torch.Size([3, 21])
o_vowels shape: torch.Size([3, 16])


In [34]:
h.view(3, -1).shape

torch.Size([3, 11664])